In [1]:
# run command 'conda install scikit-image' on project's v-env
# run command 'conda install pandas' on project's v-env

In [2]:
DEBUG_TRAIN = False
DEBUG_DATA = False

In [3]:
def encode_from_utf8(x):
    return list(map(float,bin(int(x.encode().hex(),16))[2:]))

In [4]:
def decode_from_bin(x):
    res = ""
    for i in x:
        res+=str(round(i))
        pass
    return bytearray.fromhex(hex(int(res, 2))[2:]).decode()

In [13]:
def decode_from_bin2(x):
    res = ""
    for i in x:
        res+=str(int(i>0))
        pass
    return bytearray.fromhex(hex(int(res, 2))[2:]).decode()

In [5]:
import os
import torch
import numpy as np
import pandas as pd
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader

class KoreanHandwritingDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.dataset = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.dataset.iloc[idx, 0]
        image = io.imread(img_name)
        label = self.dataset.iloc[idx, 1]
        label = np.array([label])
        sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample
    pass

In [6]:
class ToTensor(object):
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image/255.0
        return {'image': torch.from_numpy(image.reshape(1,64,64)),
                'label': torch.tensor(encode_from_utf8(label[0]))}

In [16]:
from torchvision import transforms
csv_file = "./image-data-256/labels-map.csv"
root_dir = "./image-data-256/hangul-images"
batch_size=64
learning_rate=0.002
num_epoch=100

korean_dataset = KoreanHandwritingDataset(csv_file,root_dir, transform=transforms.Compose([ToTensor()]))
dataloader = DataLoader(korean_dataset, batch_size = batch_size, shuffle = True, num_workers = 0)
#dataloader = DataLoader(korean_dataset, batch_size = batch_size, shuffle = False, num_workers = 0)

In [17]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.avgpool1 = nn.AvgPool2d(2,2) # kernel size 2x2 (32 = 64/2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2,2) # kernel size 2x2 (32 = 64/2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2,2) # kernel size 2x2 (16 = 32/2)
        self.linear1 = nn.Linear(1152,378)
        self.relu3 = nn.ReLU()
        self.linear2 = nn.Linear(378,128)
        self.relu4 = nn.ReLU()
        self.linear3 = nn.Linear(128,24)
        #self.sigmoid = nn.Sigmoid()
        pass
    def forward(self,x):
        out = self.avgpool1(x)
        if DEBUG_TRAIN : print('avgpool1: ', out.shape)
        out = self.conv1(out)
        if DEBUG_TRAIN : print('conv1: ', out.shape)
        out = self.relu1(out)
        if DEBUG_TRAIN : print('relu1: ', out.shape)
        out = self.maxpool1(out)
        if DEBUG_TRAIN : print('maxpool1: ', out.shape)
        out = self.conv2(out)
        if DEBUG_TRAIN : print('conv2: ', out.shape)
        out = self.relu2(out)
        if DEBUG_TRAIN : print('relu2: ', out.shape)
        out = self.maxpool2(out)
        if DEBUG_TRAIN : print('maxpool2: ', out.shape)
        out = out.view(out.size(0),-1)
        if DEBUG_TRAIN : print('view: ', out.shape)
        out = self.linear1(out)
        if DEBUG_TRAIN : print('linear1: ', out.shape)
        out = self.relu3(out)
        if DEBUG_TRAIN : print('relu3: ', out.shape)
        out = self.linear2(out)
        if DEBUG_TRAIN : print('linear2: ', out.shape)
        out = self.relu4(out)
        if DEBUG_TRAIN : print('relu4: ', out.shape)
        out = self.linear3(out)
        if DEBUG_TRAIN : print('linear3: ', out.shape)
        #out = self.sigmoid(out)
        #if DEBUG_TRAIN : print('sigmoid: ', out.shape)
        if DEBUG_TRAIN : print()
        return out

In [18]:
import torch.optim as optim
model=CNN().double()
loss_function=nn.BCEWithLogitsLoss()
#loss_function=nn.
optimizer= torch.optim.Adam(model.parameters(),lr=learning_rate)

> ## Training

In [19]:
import matplotlib.pyplot as plt

loss_arr=[]
for i in range(num_epoch):
    for j, sample in enumerate(dataloader):
        x = sample['image']
        y = sample['label']
        optimizer.zero_grad()
        output=model.forward(x)
        loss=loss_function(output,y)
        loss.backward()
        optimizer.step()
        if j == 0:
            if DEBUG_DATA:
                plt.figure(figsize=(32, 32))
                for k, img in enumerate(x):
                    plt.subplot(8, 8, k+1)
                    plt.imshow(img.squeeze())
                    plt.xticks([])
                    plt.yticks([])
                    pass
                plt.show()
            print('epoch: ', i+1, 'sequence: ', j, loss)
            loss_arr.append(loss.detach().numpy())

epoch:  1 sequence:  0 tensor(0.6903, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  2 sequence:  0 tensor(0.4179, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  3 sequence:  0 tensor(0.4016, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  4 sequence:  0 tensor(0.4046, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  5 sequence:  0 tensor(0.3924, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  6 sequence:  0 tensor(0.3903, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  7 sequence:  0 tensor(0.3810, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  8 sequence:  0 tensor(0.3588, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  9 sequence:  0 tensor(0.3391, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  10 sequence:  0 tensor(0.2964, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  11 sequence:  0 tensor(0.2829, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  12 sequence:  0 tensor(0.2615, grad_fn=<Bina

epoch:  96 sequence:  0 tensor(0.0002, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  97 sequence:  0 tensor(0.0002, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  98 sequence:  0 tensor(0.0002, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  99 sequence:  0 tensor(0.0001, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  100 sequence:  0 tensor(0.0001, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


> ## Testing

In [27]:
with torch.no_grad():
    socreboard=[]
    for j, sample in enumerate(dataloader):
        x = sample['image']
        y = sample['label']
        
        output = model.forward(x)
        for i in range(len(output)):
            out = decode_from_bin2(output[i].tolist())
            y_ = decode_from_bin(y[i].tolist())
            print("출력 값: ", out,
                  ", 실제 값: ", y_)
            
            if out==y_:
                scoreboard.append(1)
            else:
                scoreboard.append(0)
            #print("실제 값: ", output[i].tolist(),
            #      ", 출력 값: ", y[i].tolist())

출력 값:  전 , 실제 값:  전
출력 값:  월 , 실제 값:  월
출력 값:  밤 , 실제 값:  밤
출력 값:  파 , 실제 값:  파
출력 값:  앞 , 실제 값:  앞
출력 값:  은 , 실제 값:  은
출력 값:  군 , 실제 값:  군
출력 값:  꾸 , 실제 값:  꾸
출력 값:  가 , 실제 값:  가
출력 값:  올 , 실제 값:  올
출력 값:  간 , 실제 값:  간
출력 값:  향 , 실제 값:  향
출력 값:  력 , 실제 값:  력
출력 값:  제 , 실제 값:  제
출력 값:  재 , 실제 값:  재
출력 값:  고 , 실제 값:  고
출력 값:  방 , 실제 값:  방
출력 값:  석 , 실제 값:  석
출력 값:  외 , 실제 값:  외
출력 값:  루 , 실제 값:  루
출력 값:  오 , 실제 값:  오
출력 값:  운 , 실제 값:  운
출력 값:  변 , 실제 값:  변
출력 값:  복 , 실제 값:  복
출력 값:  프 , 실제 값:  프
출력 값:  파 , 실제 값:  파
출력 값:  년 , 실제 값:  년
출력 값:  접 , 실제 값:  접
출력 값:  물 , 실제 값:  물
출력 값:  예 , 실제 값:  예
출력 값:  발 , 실제 값:  발
출력 값:  물 , 실제 값:  물
출력 값:  세 , 실제 값:  세
출력 값:  인 , 실제 값:  인
출력 값:  군 , 실제 값:  군
출력 값:  히 , 실제 값:  히
출력 값:  신 , 실제 값:  신
출력 값:  꾸 , 실제 값:  꾸
출력 값:  과 , 실제 값:  과
출력 값:  야 , 실제 값:  야
출력 값:  속 , 실제 값:  속
출력 값:  중 , 실제 값:  중
출력 값:  소 , 실제 값:  소
출력 값:  여 , 실제 값:  여
출력 값:  손 , 실제 값:  손
출력 값:  순 , 실제 값:  순
출력 값:  잠 , 실제 값:  잠
출력 값:  부 , 실제 값:  부
출력 값:  구 , 실제 값:  구
출력 값:  점 , 실제 값:  점


출력 값:  력 , 실제 값:  력
출력 값:  름 , 실제 값:  름
출력 값:  자 , 실제 값:  자
출력 값:  회 , 실제 값:  회
출력 값:  러 , 실제 값:  러
출력 값:  력 , 실제 값:  력
출력 값:  음 , 실제 값:  음
출력 값:  극 , 실제 값:  극
출력 값:  드 , 실제 값:  드
출력 값:  예 , 실제 값:  예
출력 값:  영 , 실제 값:  영
출력 값:  길 , 실제 값:  길
출력 값:  절 , 실제 값:  절
출력 값:  래 , 실제 값:  래
출력 값:  코 , 실제 값:  코
출력 값:  들 , 실제 값:  들
출력 값:  별 , 실제 값:  별
출력 값:  울 , 실제 값:  울
출력 값:  창 , 실제 값:  창
출력 값:  확 , 실제 값:  확
출력 값:  해 , 실제 값:  해
출력 값:  자 , 실제 값:  자
출력 값:  교 , 실제 값:  교
출력 값:  물 , 실제 값:  물
출력 값:  초 , 실제 값:  초
출력 값:  소 , 실제 값:  소
출력 값:  분 , 실제 값:  분
출력 값:  우 , 실제 값:  우
출력 값:  식 , 실제 값:  식
출력 값:  송 , 실제 값:  송
출력 값:  개 , 실제 값:  개
출력 값:  양 , 실제 값:  양
출력 값:  어 , 실제 값:  어
출력 값:  배 , 실제 값:  배
출력 값:  지 , 실제 값:  지
출력 값:  건 , 실제 값:  건
출력 값:  심 , 실제 값:  심
출력 값:  복 , 실제 값:  복
출력 값:  단 , 실제 값:  단
출력 값:  건 , 실제 값:  건
출력 값:  절 , 실제 값:  절
출력 값:  바 , 실제 값:  바
출력 값:  술 , 실제 값:  술
출력 값:  료 , 실제 값:  료
출력 값:  처 , 실제 값:  처
출력 값:  처 , 실제 값:  처
출력 값:  갈 , 실제 값:  갈
출력 값:  면 , 실제 값:  면
출력 값:  이 , 실제 값:  이
출력 값:  증 , 실제 값:  증


In [31]:
sum=0
for i in range(len(scoreboard)):
    sum+=int(scoreboard[i])
performance=sum/len(scoreboard)*100
print("performance: ",performance,"%")

performance:  100.0 %
